In [46]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import OneHotEncoder
#from PIL import Image
from IPython.display import display, Image, HTML
from src.btree import BTree

In [47]:
%pip install memory-profiler --quiet
%pip install setuptools --quiet

%pip install snakeviz --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Loading the styles data

In [48]:
def process_bad_line(line):
  print(f"{line}")
  return None

styles_df = pd.read_csv('fashion-dataset/styles.csv', on_bad_lines=process_bad_line, engine='python')

['44065', 'Men', 'Personal Care', 'Fragrance', 'Perfume and Body Mist', 'Blue', 'Spring', '2017', 'Casual', 'Boss Men Perfume', ' After Shave Balm and Shower Gel Set']
['3413', 'Men', 'Apparel', 'Topwear', 'Tshirts', 'White', 'Summer', '2011', 'Casual', "Myntra Men's Yes", ' its all about me White T-shirt']
['23101', 'Men', 'Accessories', 'Accessories', 'Accessory Gift Set', 'Pink', 'Fall', '2011', 'Formal', 'Turtle Men Formal Pink Tie', ' Cufflink and Pocket Square Combo Set']
['23108', 'Men', 'Accessories', 'Accessories', 'Accessory Gift Set', 'Purple', 'Fall', '2011', 'Formal', 'Turtle Men Formal Purple Tie', ' Cufflink and Pocket Square Combo Set']
['4854', 'Unisex', 'Footwear', 'Shoes', 'Casual Shoes', 'White', 'Winter', '2012', 'Casual', 'Newfeel Unisex White', ' Blue and Orange Casual Shoes']
['23109', 'Men', 'Accessories', 'Accessories', 'Accessory Gift Set', 'Purple', 'Fall', '2011', 'Formal', 'Turtle Men Formal Purple Tie', ' Cufflink and Pocket Square Combo Set']
['23100', '

In [49]:
styles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44424 entries, 0 to 44423
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  44424 non-null  int64  
 1   gender              44424 non-null  object 
 2   masterCategory      44424 non-null  object 
 3   subCategory         44424 non-null  object 
 4   articleType         44424 non-null  object 
 5   baseColour          44409 non-null  object 
 6   season              44403 non-null  object 
 7   year                44423 non-null  float64
 8   usage               44107 non-null  object 
 9   productDisplayName  44417 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.4+ MB


In [50]:
styles_df['year'] = styles_df['year'].fillna(round(styles_df['year'].mean()))
styles_df['year'] = styles_df['year'].astype(int)
styles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44424 entries, 0 to 44423
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  44424 non-null  int64 
 1   gender              44424 non-null  object
 2   masterCategory      44424 non-null  object
 3   subCategory         44424 non-null  object
 4   articleType         44424 non-null  object
 5   baseColour          44409 non-null  object
 6   season              44403 non-null  object
 7   year                44424 non-null  int64 
 8   usage               44107 non-null  object
 9   productDisplayName  44417 non-null  object
dtypes: int64(2), object(8)
memory usage: 3.4+ MB


In [51]:
styles_df.head(5)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt


### Loading images from Styles 

In [52]:
images_df = pd.read_csv('fashion-dataset/images.csv', on_bad_lines=process_bad_line, engine='python')
images_df.head()

,filename,link
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...


In [53]:
images_df['id'] = images_df['filename'].str.replace(r'\.\w+$', '', regex=True).astype(int)
images_df.head()

,filename,link,id
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,15970
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,39386
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,59263
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...,21379
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...,53759


### Preprocesando las columnas categóricas (string)

In [54]:
# Columnas a usar en el filtrado basado en contenido
categorical_columns = ['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage']

# Usar OneHotEncoder para convertir las características categóricas
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(styles_df[categorical_columns]).toarray().astype(int)
encoded_features[0]

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [55]:
# Agregar el anio (feature no categorica) al conjunto de caracteristicas
year_feature = np.expand_dims(styles_df['year'].values, axis=1)
# Combinar las características codificadas y el año
combined_features = np.hstack((encoded_features, year_feature))

pd.set_option('display.max_columns', None)

# extraemos el array de columnas categoritas y concatenamos el anio
encoded_columns = encoder.get_feature_names_out(categorical_columns)
all_columns = np.concatenate([encoded_columns, ['year']])

# Convertimos los datos codificados a un DataFrame
product_df = pd.DataFrame(combined_features, columns=all_columns)
product_df = pd.concat([styles_df['id'], product_df], axis=1)
product_df.head(5)

,id,gender_Boys,gender_Girls,gender_Men,gender_Unisex,gender_Women,masterCategory_Accessories,masterCategory_Apparel,masterCategory_Footwear,masterCategory_Free Items,masterCategory_Home,masterCategory_Personal Care,masterCategory_Sporting Goods,subCategory_Accessories,subCategory_Apparel Set,subCategory_Bags,subCategory_Bath and Body,subCategory_Beauty Accessories,subCategory_Belts,subCategory_Bottomwear,subCategory_Cufflinks,subCategory_Dress,subCategory_Eyes,subCategory_Eyewear,subCategory_Flip Flops,subCategory_Fragrance,subCategory_Free Gifts,subCategory_Gloves,subCategory_Hair,subCategory_Headwear,subCategory_Home Furnishing,subCategory_Innerwear,subCategory_Jewellery,subCategory_Lips,subCategory_Loungewear and Nightwear,subCategory_Makeup,subCategory_Mufflers,subCategory_Nails,subCategory_Perfumes,subCategory_Sandal,subCategory_Saree,subCategory_Scarves,subCategory_Shoe Accessories,subCategory_Shoes,subCategory_Skin,subCategory_Skin Care,subCategory_Socks,subCategory_Sports Accessories,subCategory_Sports Equipment,subCategory_Stoles,subCategory_Ties,subCategory_Topwear,subCategory_Umbrellas,subCategory_Vouchers,subCategory_Wallets,subCategory_Watches,subCategory_Water Bottle,subCategory_Wristbands,articleType_Accessory Gift Set,articleType_Baby Dolls,articleType_Backpacks,articleType_Bangle,articleType_Basketballs,articleType_Bath Robe,articleType_Beauty Accessory,articleType_Belts,articleType_Blazers,articleType_Body Lotion,articleType_Body Wash and Scrub,articleType_Booties,articleType_Boxers,articleType_Bra,articleType_Bracelet,articleType_Briefs,articleType_Camisoles,articleType_Capris,articleType_Caps,articleType_Casual Shoes,articleType_Churidar,articleType_Clothing Set,articleType_Clutches,articleType_Compact,articleType_Concealer,articleType_Cufflinks,articleType_Cushion Covers,articleType_Deodorant,articleType_Dresses,articleType_Duffel Bag,articleType_Dupatta,articleType_Earrings,articleType_Eye Cream,articleType_Eyeshadow,articleType_Face Moisturisers,articleType_Face Scrub and Exfoliator,articleType_Face Serum and Gel,articleType_Face Wash and Cleanser,articleType_Flats,articleType_Flip Flops,articleType_Footballs,articleType_Formal Shoes,articleType_Foundation and Primer,articleType_Fragrance Gift Set,articleType_Free Gifts,articleType_Gloves,articleType_Hair Accessory,articleType_Hair Colour,articleType_Handbags,articleType_Hat,articleType_Headband,articleType_Heels,articleType_Highlighter and Blush,articleType_Innerwear Vests,articleType_Ipad,articleType_Jackets,articleType_Jeans,articleType_Jeggings,articleType_Jewellery Set,articleType_Jumpsuit,articleType_Kajal and Eyeliner,articleType_Key chain,articleType_Kurta Sets,articleType_Kurtas,articleType_Kurtis,articleType_Laptop Bag,articleType_Leggings,articleType_Lehenga Choli,articleType_Lip Care,articleType_Lip Gloss,articleType_Lip Liner,articleType_Lip Plumper,articleType_Lipstick,articleType_Lounge Pants,articleType_Lounge Shorts,articleType_Lounge Tshirts,articleType_Makeup Remover,articleType_Mascara,articleType_Mask and Peel,articleType_Mens Grooming Kit,articleType_Messenger Bag,articleType_Mobile Pouch,articleType_Mufflers,articleType_Nail Essentials,articleType_Nail Polish,articleType_Necklace and Chains,articleType_Nehru Jackets,articleType_Night suits,articleType_Nightdress,articleType_Patiala,articleType_Pendant,articleType_Perfume and Body Mist,articleType_Rain Jacket,articleType_Rain Trousers,articleType_Ring,articleType_Robe,articleType_Rompers,articleType_Rucksacks,articleType_Salwar,articleType_Salwar and Dupatta,articleType_Sandals,articleType_Sarees,articleType_Scarves,articleType_Shapewear,articleType_Shirts,articleType_Shoe Accessories,articleType_Shoe Laces,articleType_Shorts,articleType_Shrug,articleType_Skirts,articleType_Socks,articleType_Sports Sandals,articleType_Sports Shoes,articleType_Stockings,articleType_Stoles,articleType_Suits,articleType_Sunglasses,articleType_Sunscreen,articleType_Suspenders,articleType_Sweaters,artic

### Definiendo y armando el Árbol B

In [77]:
from sortedcontainers import SortedList

class BTreeCustom:
  def __init__(self, local_btree=False, degree=4):
    self.local_btree = local_btree
    self.tree = BTree(degree=degree) if local_btree else SortedList()

  def insert(self, product_id, feature_vector):
    feature_vector = tuple(feature_vector)
    if self.local_btree:
      self.tree.insert(feature_vector, product_id)
    else:
      self.tree.add((feature_vector, product_id))

  def search_similar(self, target_vector, k=5):
    # Buscar los productos más cercanos usando la distancia euclidiana
    distances = [(self.euclidean_distance(target_vector, np.array(product[0])), product[1])
                 for product in self.tree]
    distances.sort(key=lambda x: x[0]) # Ordenamos por la distancia más corta
    return distances[:k] # Retornar los k productos más similares

  @staticmethod
  def euclidean_distance(v1, v2):
    return sum((a - b) ** 2 for a, b in zip(v1, v2)) ** 0.5

#btree = BTreeCustom(local_btree=True, degree=4)
btree = BTreeCustom()


In [57]:
# class BTreeCustom:
#   def __init__(self):
#     self.tree = BTree(degree=4)

#   def insert(self, product_id, feature_vector):
#     feature_vector = tuple(feature_vector)
#     self.tree.insert(feature_vector, product_id)

#   # print three
#   def print_tree(self):
#     i = 0
#     for product in self.tree:
#       print(len(product))
#       print(product)
#       if i == 2:
#         break
#       i += 1

#   def search_similar(self, target_vector, k=5):
#     # Buscar los productos más cercanos usando la distancia euclidiana
#     distances = [(self.euclidean_distance(target_vector, np.array(product[0])), product[1])
#                 for product in self.tree]
#     distances.sort(key=lambda x: x[0]) # Ordenamos por la distancia más corta
#     return distances[:k] # Retornar los k productos más similares

#   @staticmethod
#   def euclidean_distance(v1, v2):
#     return sum((a - b) ** 2 for a, b in zip(v1, v2)) ** 0.5

# btree = BTreeCustom()


Insertando los features en el arbol

In [58]:
#%load_ext memory_profiler
#%load_ext snakeviz
%reload_ext snakeviz

In [78]:
import cProfile
import time


def fill_btree():
  init = time.time()
  i = 0
  for _, row in product_df.iterrows():
    feature_vector = row.drop('id').values  # Usamos los valores codificados (sin incluir el 'id')
    btree.insert(row['id'], feature_vector)
    i += 1
  end = time.time()

  print(f"Tiempo total (seg): {round(end - init, 4)}")

#cProfile.run('fill_btree()')

# Ejecutar el profiler de memoria
#%mprun -f fill_btree fill_btree()

#cProfile.run('fill_btree()','profile_result')

#%snakeviz
fill_btree()

Tiempo total (seg): 4.7958


In [79]:
len(btree.tree)

44424

In [80]:
for i, product in enumerate(btree.tree):
    print(product[1])
    if i == 3:
        break

57608
57859
57607
57860


### Inferencia: Probando la recomendación

In [81]:
fashion_df = pd.merge(styles_df, images_df.drop(columns=['link']), on='id', how='inner')
fashion_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg
...,...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013,Casual,Gas Men Caddy Casual Shoe,17036.jpg
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011,Casual,Lotto Men's Soccer Track Flip Flop,6461.jpg
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual,Puma Men Graphic Stellar Blue Tshirt,18842.jpg
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017,Casual,Rasasi Women Blue Lady Perfume,46694.jpg


#### Cargando imagenes a demanda en el dataframe

In [82]:

def display_products(df: pd.DataFrame):
    html = ""
    for _, row in df.iterrows():
        html += f"""
            <div style="display: inline-block; text-align: center; margin: 10px;">
                <img src="fashion-dataset/images/{row['filename']}" width="150" /><br>
                <b style="width=180">{row['productDisplayName']}</b><br>
                Id: {row['id']} | Gender: {row['gender']}<br>
                {row['masterCategory']} | {row['subCategory']}<br>
            </div>
        """
    display(HTML(html))

In [87]:
target_product_id = 1762 #1590 #1634 #1163  # 37935
result_df = fashion_df[fashion_df['id'] == target_product_id].copy()

#result_df['image'] = result_df['filename'].apply(load_image)
#result_df
display_products(result_df)

In [88]:
start = time.time()
# Seleccionar un producto como base de recomendación
target_vector = product_df[product_df['id'] == target_product_id].values[0][1:]

# Obtener productos más similares
recommended_products = btree.search_similar(target_vector, k=8)

# Mostrar IDs de los productos recomendados
recommended_product_ids = [product[1] for product in recommended_products]
#print("Productos recomendados:", recommended_product_ids)

# Mostrar detalles de los productos recomendados
recommended_details = fashion_df[fashion_df['id'].isin(recommended_product_ids)]
end = time.time()

print(f"tiempo total (seg): {round(end - start,3)}")
recommended_details #[['id', 'articleType', 'productDisplayName']]

tiempo total (seg): 2.189


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename
3925,2289,Women,Apparel,Topwear,Tshirts,Blue,Fall,2010,Sports,Nike Women Blue T-shirt,2289.jpg
5348,1809,Women,Apparel,Topwear,Tshirts,Blue,Fall,2010,Sports,Decathlon Profilter Women Blue T-shirt,1809.jpg
6671,1986,Women,Apparel,Topwear,Tshirts,Blue,Fall,2010,Sports,ADIDAS Womens Blue V Neck T-shirt,1986.jpg
9715,9985,Women,Apparel,Topwear,Tshirts,Blue,Fall,2011,Sports,Nike Women As Jdi Pixels Blue T-Shirts,9985.jpg
10670,2309,Women,Apparel,Topwear,Tshirts,Blue,Fall,2010,Sports,Nike Women Miller Blue T-shirt,2309.jpg
15130,9964,Women,Apparel,Topwear,Tshirts,Blue,Fall,2011,Sports,Nike Women As Statement Blue T-Shirts,9964.jpg
28621,2305,Women,Apparel,Topwear,Tshirts,Blue,Fall,2010,Sports,Nike Women Blue Polo T-shirt,2305.jpg
35923,1762,Women,Apparel,Topwear,Tshirts,Blue,Fall,2010,Sports,Lotto Women Cathy Blue T-shirt,1762.jpg


In [89]:
#target_vector

In [90]:
display_products(recommended_details)